<a href="https://colab.research.google.com/github/Michwynn/London-Airbnb-Analysis---2/blob/Elias/Airbnb_Data_Base_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuration and library set-up**

In [65]:
# data manipulation
import pandas as pd
import numpy as np 
import re # regex
import random
import string
from collections import Counter, defaultdict

pattern = '\w+' # setting regex pattern

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # comment out if already downloaded
stop_words = set(stopwords.words('english'))
nltk.download('punkt')     # comment out if already downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


# machine learning
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# timeit
from tqdm import tqdm

# data visualisation
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# set up working directory
import os
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Airbnb_Milestone2

# supress warnings
import warnings 
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)

# NLP
!pip install pycld2
import pycld2 as cld2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1wUOfFY-ki2nFzneeaTtXLEeMjaSdKrrj/Airbnb_Milestone2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# read data
reviews_df = pd.read_csv('Datasets/reviews.csv') 
# remove white spaces in column headings
reviews_df.columns = reviews_df.columns.str.strip() 
# Examine dataset
display(reviews_df.head(10))
display(reviews_df.shape)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator… he was very hel...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.<br/>Good l...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
5,52228441,488275073602903479,2021-11-04,37185586,Ruslans,The flat is very well connected to the city by...
6,17501535,668631693343526023,2022-07-11,5132271,Marc,Nice place
7,52228441,588257361951713358,2022-03-22,248672143,Sveinung,Big apartment but a little bit run down. Kitc...
8,17501535,633193507110858827,2022-05-23,413544004,Daniel,"Amazing place, we loved it. Tons of space, nic..."
9,52228441,491866989310660615,2021-11-09,8452946,Johnny,"Duccio was a great host, super friendly with l..."


(1216212, 6)

# **Data Cleaning**

**Steps to follow**

1.   Drop rows containing NAN values (DONE)
2.   Drop rows with empty string for 'comments' column (DONE)
3.   Remove HTML tags from string in 'comments' column (DONE)
4.   Remove special characters including emojis from strings (DONE)
5.   Drop rows containing non-english string in 'comments' column (DONE)
6.   Remove stopwords from string in 'comments' column
7.   Remove any short-word form string in 'comments' column






In [4]:
# check if there is any null value 
reviews_df.isnull().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     2
comments         92
dtype: int64

In [5]:
# check if there is any empty string in the comments column
reviews_df.loc[reviews_df["comments"] == "."]

,listing_id,id,date,reviewer_id,reviewer_name,comments
92,21526298,675869744229404323,2022-07-21,215666012,David,.
319,32537244,572622781,2019-12-03,310625523,Fernando,.
2388,3370968,551274991374749641,2022-01-30,88466671,Bekee,.
3143,13396482,562849095,2019-11-11,299309771,Cecilie,.
3334,41509,49170962,2015-10-01,736090,Alberto,.
...,...,...,...,...,...,...
1209963,31495400,518304382,2019-08-26,34520842,Augusto,.
1212428,31201633,419606016,2019-03-04,243989008,Javier,.
1212852,31304270,419096711,2019-03-03,244943897,Bernard,.
1214589,32002019,474417589,2019-06-23,64631352,Margaret,.


In [6]:
# Drop the null value and comments value with "."
reviews_df = reviews_df[reviews_df['comments'].notna()]
display(reviews_df.shape)

(1216120, 6)

---
92 Comments containing null value dropped

In [7]:
# Drop the rows with empty string for comments column
reviews_df = reviews_df.loc[reviews_df["comments"] != "."]
display(reviews_df.shape)

(1214643, 6)

---
1477 Comments containing empty string dropped

In [8]:
# helper function to remove HTML tags from comments
from bs4 import BeautifulSoup
copy_df = reviews_df.copy()
#small_df = copy_df.iloc[:100,]
copy_df['comments'] = copy_df['comments'].apply(lambda s: BeautifulSoup(s).text)
display(copy_df.head(100))

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator… he was very hel...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
...,...,...,...,...,...,...
96,17501535,588209695468203799,2022-03-22,31576937,Julian,"Superb hospitality. Dominic was friendly, resp..."
97,32537244,617521403,2020-03-13,84592669,Bun-Hok,"Good place to stay, although a bit noisy, as i..."
98,17501535,594341871,2020-01-18,84447796,Mallory,Easy and convenient check-in/check-out. The s...
99,17501535,561415884152066004,2022-02-13,147957618,Sandra,Dominic was a great host! We had to reach out ...


In [9]:
# removing special characters including punctuations and emojis. 
copy2_df = copy_df.copy()

copy2_df['comments'] = copy2_df['comments'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
copy2_df.columns = copy2_df.columns.str.strip() 

display(copy2_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
...,...,...,...,...,...,...
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...
1216208,14832630,173531056,2017-07-23,130284505,Véronique,Excellente situation au plein centre de Londre...
1216209,14832630,153170957,2017-05-20,119296298,Nicole,"Todo perfecto Ubicación, comodidad y Rachel c..."
1216210,14832630,145456398,2017-04-18,16394435,Christophe,Tout dabord même si nous navons pas eu le plai...


# **Comments Language Identification**

In [10]:
# installing and Importing fasttext library for language identification
!pip install fasttext
import fasttext
# settting up the pretrained model
fasttxt_model = fasttext.load_model("lid.176.bin")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 6.8 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165347 sha256=a5e8442bc021b15751020928400d8d7ed500848733496ddeaab36e6139f903bd
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [11]:
# test model with example text
fasttxt_model.predict('이 집은 정말 좋아요! 맘에 쏙 드네요')[0][0][-2:]

'ko'

In [12]:
# Adding an extra column for language identification
copy2_df['lang'] = copy2_df['comments'].apply(lambda x: fasttxt_model.predict(x)[0][0][-2:])
display(copy2_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en
...,...,...,...,...,...,...,...
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...,en
1216208,14832630,173531056,2017-07-23,130284505,Véronique,Excellente situation au plein centre de Londre...,fr
1216209,14832630,153170957,2017-05-20,119296298,Nicole,"Todo perfecto Ubicación, comodidad y Rachel c...",es
1216210,14832630,145456398,2017-04-18,16394435,Christophe,Tout dabord même si nous navons pas eu le plai...,fr


In [13]:
# calculating number of rows with non-english comments 
display(copy2_df.lang.value_counts())
prop = (1214643-1089747)/1214643*100
f'The total non-english comments proportion is {prop}.'

en    1089747
fr      45182
es      22556
de      16384
it      10215
       ...   
br          1
mk          1
ne          1
io          1
ms          1
Name: lang, Length: 77, dtype: int64

'The total non-english comments proportion is 10.282527458685392.'

---
**The total non-english comments proportion is 10.28%. 
Therefore, it is safe to drop the rows with non-english comments.**




In [31]:
# keeping only rows with English comments
copy3_df = copy2_df[copy2_df['lang']=='en']
display(copy3_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en
...,...,...,...,...,...,...,...
1216194,14832630,107089577,2016-10-09,46576370,Anders,"This was an excellent apartment. Very clean, w...",en
1216195,14832630,105494942,2016-10-02,31582062,Magda,The place was a dream come true\r Its newly re...,en
1216196,14832630,104922701,2016-09-29,95543679,Joe,Her place is so greatNice and clean. My wife a...,en
1216197,14832630,102192086,2016-09-17,16267738,Neta,Right at the heart of Soho.The place is comfor...,en


In [15]:
# Special case that has both chinese and English
display(copy3_df.loc[[1216207]])

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
1216207,14832630,184884203,2017-08-20,54407484,丽云,干净舒适适合家庭入住地段优越景点全部很多都可以步行到达节省了交通费房间设施齐备能满足我们的一...,en


In [32]:
# drop rows containing multiple languages
copy3_df['Deteced_lang'] = copy3_df['comments'].map(lambda a: cld2.detect(a,bestEffort=True)[2][1][0])
copy3_df = copy3_df[copy3_df['Deteced_lang']=='Unknown']
#copy4_df.comments.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
#display(copy3_df)


In [33]:
# drop rows containing non-meaningful words
copy3_df['Deteced_lang'] = copy3_df['comments'].map(lambda a: cld2.detect(a,bestEffort=True)[2][0][0])
copy3_df = copy3_df[copy3_df['Deteced_lang']!='Unknown']

In [34]:
copy3_df = copy3_df.drop('Deteced_lang',axis=1)
display(copy3_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en
...,...,...,...,...,...,...,...
1216193,14832630,110143844,2016-10-24,45223954,Ellen,We had a really great stay in Rachels central ...,en
1216194,14832630,107089577,2016-10-09,46576370,Anders,"This was an excellent apartment. Very clean, w...",en
1216195,14832630,105494942,2016-10-02,31582062,Magda,The place was a dream come true\r Its newly re...,en
1216196,14832630,104922701,2016-09-29,95543679,Joe,Her place is so greatNice and clean. My wife a...,en


In [37]:
# Data pre-processing using conventional method (ok accuracy / fast speed)
def clean_text(comments):
  punc = """!()-![]{};:,+'"\,<>./?@#$%^&*_~Â"""
  punc_removed = [char for char in comments if char not in punc]
  punc_join = ''.join(punc_removed)
  cleaned = []
  splitted = comments.split()
  for x in splitted:
    if x.isalpha() and x not in stop_words:
      cleaned.append(x.lower())
  result = ' '.join(cleaned)
  return result

In [38]:
%%time
copy3_df['cleaned_comments'] = copy3_df['comments'].apply(clean_text)

CPU times: user 26.7 s, sys: 320 ms, total: 27.1 s
Wall time: 27.1 s


In [39]:
display(copy3_df)

,listing_id,id,date,reviewer_id,reviewer_name,comments,lang,cleaned_comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi...",en,great host responsive
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...,en,duccio lovely friendly from arrival instructio...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator he was very help...,en,duccio good communicator helpful making stay g...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.Good locati...,en,not entirely compliant location basic shower w...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host,en,great place great host
...,...,...,...,...,...,...,...,...
1216193,14832630,110143844,2016-10-24,45223954,Ellen,We had a really great stay in Rachels central ...,en,we really great stay rachels central clean wel...
1216194,14832630,107089577,2016-10-09,46576370,Anders,"This was an excellent apartment. Very clean, w...",en,this excellent very well equipped location par...
1216195,14832630,105494942,2016-10-02,31582062,Magda,The place was a dream come true\r Its newly re...,en,the place dream come true its newly even flowe...
1216196,14832630,104922701,2016-09-29,95543679,Joe,Her place is so greatNice and clean. My wife a...,en,her place greatnice my wife wonderful


In [55]:
# Data pre-processing using spacy (great accuracy / slow speed)
import spacy
#!python -m spacy download en_core_web_trf
nlp = spacy.load("en_core_web_sm",disable=['tagger', 'parser', 'ner'])
def spacyClean(doc,stemming = False):
    #doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    return " ".join(final_token)

In [56]:
%%time
copy3_df['cleaned_comments_spacy'] = copy3_df['comments'].apply(spacyClean)

KeyboardInterrupt: ignored

In [61]:
def clean_up(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im','i']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [ ]:
copy3_df['cleaned_comments_spacy'] = copy3_df['comments'].apply(clean_up,stem='Lem')